# Weeks 3 & 4 Milestone #2

- Joshua Burden
- Bellevue University
- DSC540 Data Preparation
- Cathrine Williams
- 07/02/2022


In [48]:
# Import Libraries and Modules used for class
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import lxml
import seaborn
import statsmodels
import patsy
import requests
from datetime import datetime, timedelta

In [49]:
covid_df = pd.read_csv('country_wise_latest.csv')
covid_df1 = pd.read_csv('vbim-akqf.csv')

In [50]:
covid_df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [51]:
covid_df1.head()

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
0,2021-12-31T00:00:00.000,2021-12-31T00:00:00.000,NaN,NaN,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Missing,Missing,Missing,Missing
1,2020-03-21T00:00:00.000,2020-03-30T00:00:00.000,NaN,2020-03-21T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Missing,Missing,Missing,Missing
2,2020-04-30T00:00:00.000,2020-05-05T00:00:00.000,NaN,NaN,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Missing,Missing,Missing,Missing
3,2021-04-27T00:00:00.000,2021-05-04T00:00:00.000,NaN,2021-04-27T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,Missing,No,Missing
4,2021-01-25T00:00:00.000,2021-01-28T00:00:00.000,NaN,2021-01-25T00:00:00.000,Probable Case,Male,80+ Years,"White, Non-Hispanic",No,Missing,No,Missing


##### First tranformation

In [52]:
# covid_df.drop(labels=['WHO Region'],axis=1,inplace=True)
covid_df.drop(labels=['1 week % increase'],axis=1,inplace=True)
covid_df.drop(labels=['Confirmed last week'],axis=1,inplace=True)
covid_df.drop(labels=['1 week change'],axis=1,inplace=True)


In [53]:
covid_df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,Africa


##### second transformation

In [45]:
covid_df1 = covid_df1.rename(columns={'age_group':'Age Group',
                                      'cdc_case_earliest_dt':'CDC Case Earliest Date Time', 
                                      'cdc_report_dt':'CDC Report Date Time',
                                     'pos_spec_dt':'POS Spec Date Time',
                                     'onset_dt':'Onset Date Time',
                                     'current_status':'Current Status',
                                     'age_group':'Age Group',
                                     'race_ethnicity_combined':'Race/Ethnicity Combined',
                                     'hosp_yn':'Hospitalization',
                                     'icu_yn':'ICU',
                                     'death_yn':'Death',
                                     'medcond_yn':'Medical Condition'})

In [46]:
covid_df1.head()

,CDC Case Earliest Date Time,CDC Report Date Time,POS Spec Date Time,Onset Date Time,Current Status,sex,Age Group,Race/Ethnicity Combined,Hospitalization,ICU,Death,Medical Condition
7,2021-02-15T00:00:00.000,2022-03-22T00:00:00.000,2021-02-22T00:00:00.000,2021-02-15T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",No,Missing,No,Yes
22,2021-01-02T00:00:00.000,2021-01-15T00:00:00.000,2021-01-06T00:00:00.000,2021-01-02T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,Missing,No,Missing
32,2021-03-24T00:00:00.000,2021-12-03T00:00:00.000,2021-03-24T00:00:00.000,2021-03-24T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,No,Yes,Yes
43,2021-01-14T00:00:00.000,2022-02-26T00:00:00.000,2021-01-14T00:00:00.000,2021-01-14T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",No,Missing,No,Yes
51,2022-01-01T00:00:00.000,2022-01-14T00:00:00.000,2022-01-11T00:00:00.000,2022-01-01T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,Missing,Missing,Missing


##### Third Tranformation

In [32]:
covid_df = covid_df.rename(columns={'Country/Region':'Geographical Location','Active':'Current Active Cases #'})

In [33]:
covid_df.head()

,Geographical Location,Confirmed,Deaths,Recovered,Current Active Cases #,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,Africa


##### Fourth Transformation

In [36]:
covid_df1.isnull().sum()

cdc_case_earliest_dt         0
cdc_report_dt               56
pos_spec_dt                723
onset_dt                   559
current_status               0
sex                          0
age_group                    0
race_ethnicity_combined      0
hosp_yn                      0
icu_yn                       0
death_yn                     0
medcond_yn                   0
dtype: int64

In [37]:
# had to use the covid_df1 because the cleaned data had no null values found

covid_df1 = covid_df1[covid_df1['cdc_report_dt'].notna()]
covid_df1 = covid_df1[covid_df1['pos_spec_dt'].notna()]
covid_df1 = covid_df1[covid_df1['onset_dt'].notna()]

In [38]:
covid_df1.head()

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
7,2021-02-15T00:00:00.000,2022-03-22T00:00:00.000,2021-02-22T00:00:00.000,2021-02-15T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",No,Missing,No,Yes
22,2021-01-02T00:00:00.000,2021-01-15T00:00:00.000,2021-01-06T00:00:00.000,2021-01-02T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,Missing,No,Missing
32,2021-03-24T00:00:00.000,2021-12-03T00:00:00.000,2021-03-24T00:00:00.000,2021-03-24T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,No,Yes,Yes
43,2021-01-14T00:00:00.000,2022-02-26T00:00:00.000,2021-01-14T00:00:00.000,2021-01-14T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",No,Missing,No,Yes
51,2022-01-01T00:00:00.000,2022-01-14T00:00:00.000,2022-01-11T00:00:00.000,2022-01-01T00:00:00.000,Laboratory-confirmed case,Male,80+ Years,"White, Non-Hispanic",Yes,Missing,Missing,Missing


In [39]:
covid_df1.isnull().sum()

cdc_case_earliest_dt       0
cdc_report_dt              0
pos_spec_dt                0
onset_dt                   0
current_status             0
sex                        0
age_group                  0
race_ethnicity_combined    0
hosp_yn                    0
icu_yn                     0
death_yn                   0
medcond_yn                 0
dtype: int64

In [34]:
covid_df.isnull().sum()

Geographical Location     0
Confirmed                 0
Deaths                    0
Recovered                 0
Current Active Cases #    0
New cases                 0
New deaths                0
New recovered             0
Deaths / 100 Cases        0
Recovered / 100 Cases     0
Deaths / 100 Recovered    0
WHO Region                0
dtype: int64

##### Fifth Transformation

In [40]:
covid_df = covid_df[[ 'Geographical Location',                     
 'WHO Region',
 'Current Active Cases #',                 
 'Confirmed',
 'New cases',
 'Deaths',
 'New deaths',
 'Recovered',            
 'New recovered',
 'Deaths / 100 Cases',
 'Recovered / 100 Cases',                 
 'Deaths / 100 Recovered'
]]

In [41]:
covid_df.head()

,Geographical Location,WHO Region,Current Active Cases #,Confirmed,New cases,Deaths,New deaths,Recovered,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered
0,Afghanistan,Eastern Mediterranean,9796,36263,106,1269,10,25198,18,3.50,69.49,5.04
1,Albania,Europe,1991,4880,117,144,6,2745,63,2.95,56.25,5.25
2,Algeria,Africa,7973,27973,616,1163,8,18837,749,4.16,67.34,6.17
3,Andorra,Europe,52,907,10,52,0,803,0,5.73,88.53,6.48
4,Angola,Africa,667,950,18,41,1,242,0,4.32,25.47,16.94
